In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json

In [2]:
def getdata(url):
	r = requests.get(url)
	return r.text

In [3]:
file_path = '.'

In [4]:
website = "https://ait.ac.th/"

In [5]:
dict_href_links = {}
pdf_links = set()

def get_links(website_link):
	html_data = getdata(website_link)
	soup = BeautifulSoup(html_data, "html.parser")
	list_links = []
	for link in soup.find_all("a", href=True):

		link_str = str(link["href"])

		if link_str.startswith((str(website))):
			list_links.append(link["href"])

		# Include all href that do not start with website link but with "/"
		if link_str.startswith("/"):
			if link["href"] not in dict_href_links:
				# print(link["href"])
				dict_href_links[link["href"]] = None
				link_with_www = website_link + link["href"][1:]
				# print("adjusted link =", link_with_www)
				list_links.append(link_with_www)
		
		if link_str.endswith(".pdf"):
			if link_str.startswith("/"):
				file_link_with_www = website_link + link["href"][1:]
			else:
				file_link_with_www = link_str

			if file_link_with_www not in pdf_links:
				pdf_response = requests.get(file_link_with_www)
				with open(f"{file_path}/data/pdf/{file_link_with_www.split('/')[-1]}", 'wb') as file:
					file.write(pdf_response.content)
				pdf_links.add(file_link_with_www)

	# Convert list of links to dictionary and define keys as the links and the values as "Not-checked"
	dict_links = dict.fromkeys(list_links, "Not-checked")
	return dict_links

In [6]:
def get_subpage_links(l):
	for link in tqdm(l):
		# If not crawled through this page start crawling and get links
		if l[link] == "Not-checked":
			dict_links_subpages = get_links(link)
			# Change the dictionary value of the link to "Checked"
			l[link] = "Checked"
		else:
			# Create an empty dictionary in case every link is checked
			dict_links_subpages = {}
		# Add new dictionary to old dictionary
		l = {**dict_links_subpages, **l}
	return l

In [7]:
# add websuite WITH slash on end
website = "https://ait.ac.th/"
# create dictionary of website
dict_links = {website:"Not-checked"}

counter, counter2 = None, 0
while counter != 0:
	counter2 += 1
	dict_links2 = get_subpage_links(dict_links)
	# Count number of non-values and set counter to 0 if there are no values within the dictionary equal to the string "Not-checked"
	# https://stackoverflow.com/questions/48371856/count-the-number-of-occurrences-of-a-certain-value-in-a-dictionary-in-python
	counter = sum(value == "Not-checked" for value in dict_links2.values())
	# Print some statements
	print("")
	print("THIS IS LOOP ITERATION NUMBER", counter2)
	print("LENGTH OF DICTIONARY WITH LINKS =", len(dict_links2))
	print("NUMBER OF 'Not-checked' LINKS = ", counter)
	print("")
	dict_links = dict_links2
	# Save list in json file
	a_file = open("links.json", "w")
	json.dump(dict_links, a_file)
	a_file.close()

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.20s/it]



THIS IS LOOP ITERATION NUMBER 1
LENGTH OF DICTIONARY WITH LINKS = 71
NUMBER OF 'Not-checked' LINKS =  70



100%|██████████| 71/71 [02:02<00:00,  1.73s/it]



THIS IS LOOP ITERATION NUMBER 2
LENGTH OF DICTIONARY WITH LINKS = 448
NUMBER OF 'Not-checked' LINKS =  377



 17%|█▋        | 78/448 [01:54<09:03,  1.47s/it]


MissingSchema: Invalid URL '/wp-content/uploads/2022/12/DFAB-Broucher-Updated-28.10.2022.pdf': No scheme supplied. Perhaps you meant https:///wp-content/uploads/2022/12/DFAB-Broucher-Updated-28.10.2022.pdf?